# Object Oriented Programming (OOP) in Python - for Beginers/Intermediate (Part 2)

Οbject Oriented Programming (OOP) for Crypto lovers

In this second part of our series on Object-Oriented Programming (OOP) in Python for Beginners, we take a deeper dive into practical OOP concepts. Building upon the knowledge gained in the first part, we introduce the creation of a specialized class known as CryptoAnalysisClass. This class serves as a tool for effectively retrieving time series cryptocurrency data from CoinGecko, a prominent cryptocurrency data aggregator.With CryptoAnalysisClass, we'll explore the process of retrieving and organizing data, focusing on coins like "tether" and "bitcoin." The class doesn't stop there; it also excels in data cleaning and preprocessing. It ensures data integrity by converting cryptocurrency prices into a numeric format, eliminating duplicate records, and intelligently filling in missing values.

**The code below might seem intimidating and complex, but don't be afraid. In this tutorial, we are going to build our knowledge step by step**


If you enjoy reading stories of this nature and wish to show your support for my writing, you may contemplate becoming a Medium member. By subscribing for just $5 per month, you’ll gain boundless entry to a vast collection of Python guides and Data science articles. Additionally, if you choose to sign up through my referral link, I’ll receive a modest commission at no extra cost to you.

## CryptoAnalysisClass the Attributes
The `CryptoAnalysisClass` is a Python class, designed to facilitate the analysis of cryptocurrency market data using the CoinGecko API. This class provides a structured and reusable way to obtain a list of crypto coins from a specified exchange, such as Binance, and store them for further analysis.  

However, we can modify the class to work with other exchanges (such as Kraken, Bybit_spot, Kucoin...) by allowing the exchange name to be provided as a parameter when creating an instance of the class. To retrieve the list of available exchanges from CoinGecko, you can run the following code.

I have prepared a Jupyter Notebook to accompany this blog post, which can be viewed on my GitHub.

In [1]:
# Import the necessary libraries
import pandas as pd  # Import pandas library for data manipulation
import time  # Import time library for time-related functions
# Set the display option for floating-point formatting
pd.set_option('display.float_format', '{:.2f}'.format)
from pycoingecko import CoinGeckoAPI  # Import the CoinGeckoAPI class from the pycoingecko library

# Create an instance of the CoinGeckoAPI class
cg = CoinGeckoAPI()

# Retrieve the list of exchanges from CoinGecko and extract the 'id' column
exchange_list = pd.DataFrame(cg.get_exchanges_list())["id"]

# Display the first 5 exchange IDs from the list
print(list(exchange_list)[:5]) #by removing the # (uncomment the line) we can obtain all the available exchanges

['binance', 'bybit_spot', 'gdax', 'huobi', 'kraken']


# Initializing the Cryptocurrency Market Data Analysis Class
In this tutorial we are going to use (as default parameter) the Binance exchange. In the first part of our class `CryptoAnalysisClassCryptoAnalysisClass` we define: 
- `class CryptoAnalysisClass()`: This line defines a class named CryptoAnalysisClass. When defining a class, we start with the keyword 'class,' which is highlighted in green, indicating it's a keyword. We follow this with our desired class name, like `CryptoAnalysisClass`. However, there are rules and conventions to follow. White spaces are not allowed in class names. When a class name consists of multiple words, avoid using separators like underscores. Instead, adhere to the convention of title case, where each word begins with an uppercase letter. 

- `def __init__(self, exchange="binance")`: **This is a special method called the constructor**, which is executed when an object of the class is created and initializes the object's attributes. The self parameter refers to the instance of the class that is being created. **The exchange="binance" is a default parameter**, meaning if no exchange argument is provided when creating an instance of the class, it will default to "binance." We can provide the exchange name as a string, enclosed in quotation in order to replace the deafualt excahneg Biannce (see figure below)

- self.exchange = exchange: This line sets the exchange attribute of the class instance (object) to the value provided as an argument or the default value "binance."

- self.cg = CoinGeckoAPI(): This line creates an instance of the CoinGeckoAPI class from the pycoingecko library. It will be used to interact with the CoinGecko API to retrieve cryptocurrency data.

- self.coins = []: This line initializes an empty list named coins. This list will be used to store cryptocurrency coin information retrieved from the API (see below).

- self.df2 = None: This line initializes an attribute named df2 to None.  This attribute will be used to store a DataFrame, likely for further data analysis (see below).

In [2]:
import pandas as pd
import time
from pycoingecko import CoinGeckoAPI
class CryptoAnalysisClass():
    """
    A class for analyzing cryptocurrency market data using the CoinGecko API [1].

    Attributes:
        exchange (str): The cryptocurrency exchange to retrieve data from (default is "binance").
        cg (CoinGeckoAPI): An instance of the CoinGeckoAPI class to interact with the CoinGecko API.
        coins (list): An empty list to store cryptocurrency coin IDs (see code below). 
        df2 (pd.DataFrame): An empty DataFrame for storing analysis results (see code below).

    """

    def __init__(self, exchange="binance"):
        """
        Initializes a CryptoAnalysisClass object with the provided exchange.

        Args:
            exchange (str, optional): The cryptocurrency exchange to use (default is "binance").
        """
        self.exchange = exchange
        self.cg = CoinGeckoAPI()
        self.coins = [] #Initialize an empty list 
        self.df2 = None  # Initialize an empty DataFrame

![](class1.jpg)

## Retrieving Coin List

We are going to add some functionality to our class. The coin_list method shown below is designed to retrieve a list of cryptocurrency coins that can be traded with "tether" and "bitcoin" from the specified cryptocurrency exchange (by default is "binance"). It utilizes the `self.cg.get_exchanges_tickers_by_id(id=self.exchange, page=page)["tickers"]` (see figure below) code to retrieve cryptocurrency data from the CoinGecko API. This data includes information about various coins and their trading pairs on the specified exchange. **The reason for selecting "tether" and "bitcoin" is  because these two cryptocurrencies are commonly used as base currencies for trading a wide range of other cryptocurrencies.** This approach simplifies the list and makes it more relevant for most users interested in cryptocurrency trading analysis.

In [3]:
import pandas as pd
import time
from pycoingecko import CoinGeckoAPI
class CryptoAnalysisClass():
    """
    A class for analyzing cryptocurrency market data using the CoinGecko API [1].

    Attributes:
        exchange (str): The cryptocurrency exchange to retrieve data from (default is "binance").
        cg (CoinGeckoAPI): An instance of the CoinGeckoAPI class to interact with the CoinGecko API.
        coins (list): An empty list to store cryptocurrency coin IDs (see code below). 
        df2 (pd.DataFrame): An empty DataFrame for storing analysis results (see code below).

    """

    def __init__(self, exchange="binance"):
        """
        Initializes a CryptoAnalysisClass object with the provided exchange.

        Args:
            exchange (str, optional): The cryptocurrency exchange to use (default is "binance").
        """
        self.exchange = exchange
        self.cg = CoinGeckoAPI()
        self.coins = [] #Initialize an empty list 
        self.df2 = None  # Initialize an empty DataFrame
        
    def coin_list(self): 
        """
    Retrieve a list of cryptocurrency coins from the specified exchange.

    This method retrieves cryptocurrency data from the CoinGecko API for the specified exchange
    and filters the coins to include only "tether" and "bitcoin".

    Args:
        None

    Returns:
        List of coins that can be traded with "tether" and "bitcoin". 

    Example:
        To fetch and filter coins, call this method as follows:
        >>> crypto_analyzer = CryptoAnalysisClass()
        >>> crypto_analyzer.coin_list()
        Download Completed
    """
        page = 1
        df = pd.DataFrame()  # Initialize an empty DataFrame

        while True:
            # Retrieve ticker data for the specified cryptocurrency exchange (self.exchange) and page number (page)
            data = self.cg.get_exchanges_tickers_by_id(id=self.exchange, page=page)["tickers"]

            if not data:
                print("Download of coin list Completed")
                break
            else:
                # Concatenate DataFrames
                df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
                print(f'Downloading Data...: {page}', end="\r")
                page += 1
                time.sleep(1.5)
                
        # Filter the DataFrame 'df' to include only rows where 'target_coin_id' is 'tether' or 'bitcoin'
        coins_df = df.loc[df["target_coin_id"].isin(["tether", "bitcoin"])]
        
        # Extract the 'coin_id' column from the filtered DataFrame and convert it to a list
        self.coins = list(coins_df["coin_id"])
        
        # Clear the printed line
        print(" " * len("Downloading Data...: 999"), end="\r")
        return self.coins

![](class2.jpg)

In [4]:
# Create an instance of the CryptoAnalysisClass called my_crypto
my_crypto = CryptoAnalysisClass()

# Call the coin_list method to retrieve a list of cryptocurrency coins [and display (for practical reasons) the first 5 elements]
my_crypto.coin_list()[:5]

Download of coin list Completed


['usd-coin', 'true-usd', 'bitcoin', 'ethereum', 'first-digital-usd']

## Retrieving Panel Data

Next, we will introduce additional functionality, specifically a method for retrieving panel data for the list of cryptocurrencies obtained earlier. Panel data, also known as longitudinal data or cross-sectional time-series data, is a type of dataset that combines elements of both cross-sectional data and time-series data.  

- **Cross-Sectional Dimension**: Panel data includes measurements for various cryptocurenceis
- **Time-Series Dimension**: Panel data also includes observations for each of the cross-sectional units (for each cryptocurency) at multiple points in time.  

*In essence, panel data captures information about multiple cryptocurencies over multiple time periods.*


We are going to use the `panel_data method`, which accepts an optional argument max_coins, specifying the maximum number of cryptocurrencies to retrieve data for. By default, it is set to 5, meaning it will fetch data for the first 5 coins in the self.coins list.

However, if you wish to download panel data for all the cryptocurrencies in the self.coins list (please note that this might take a considerable amount of time, several minutes or more), you can determine the total number of cryptocurrencies stored in the self.coins list by executing the following code:

- `downloaded_coins = crypto_analyzer.coin_list()`: Call the coin_list() method to retrieve and filter coins
- `num_downloaded_coins = len(downloaded_coins)`: Check the number of downloaded coins


And then we can use `num_downloaded_coins` as the value for the `max_coins` parameter when calling the panel_data() method to download data for all available cryptocurrencies.


In [5]:
# Call the coin_list() method to retrieve and filter coins
downloaded_coins = my_crypto.coin_list()

# Check the number of downloaded coins
num_downloaded_coins = len(downloaded_coins)

print(num_downloaded_coins)

Download of coin list Completed
638                     


The `panel_data method` first checks if the self.coins list is empty. If the list is empty, it prints a message indicating that it's calling the coin_list() method to populate it. This ensures that there are cryptocurrencies to analyze.  

It then enters a loop to iterate through the first max_coins cryptocurrencies (by default here, 5) in the self.coins list. For each coin:

- attempts to retrieve time series data using the `self.cg.get_coin_market_chart_by_id()` method, specifying parameters such as the cryptocurrency ID, currency for pricing ("eur" for Euros), the time period ("max" for maximum available), and data interval ("daily").  

- extracts the "Date," "Prices," and "Market Caps" columns from df3 and performs data transformations such as converting the "Date" column to datetime format and adding a new "Symbol" column to indicate the cryptocurrency symbol.  

- converts the "Date" column to a datetime format.
- removes the last row of df3, which is done to handle inconsistency in the data source.
- concatenates df3 with the existing df2 DataFrame.
- assigns the concatenated DataFrame df2 to the class attribute `self.df2` (see section above: `Initializing the Cryptocurrency Market Data Analysis Class`).
- Finally, it returns the df2 DataFrame, which contains the time series data for the specified cryptocurrencies

*If an exception occurs during data retrieval, it prints an error message.*

In [6]:
import pandas as pd
import time
from pycoingecko import CoinGeckoAPI
class CryptoAnalysisClass():
    """
    A class for analyzing cryptocurrency market data using the CoinGecko API [1].

    Attributes:
        exchange (str): The cryptocurrency exchange to retrieve data from (default is "binance").
        cg (CoinGeckoAPI): An instance of the CoinGeckoAPI class to interact with the CoinGecko API.
        coins (list): An empty list to store cryptocurrency coin IDs (see code below). 
        df2 (pd.DataFrame): An empty DataFrame for storing analysis results (see code below).

    """

    def __init__(self, exchange="binance"):
        """
        Initializes a CryptoAnalysisClass object with the provided exchange.

        Args:
            exchange (str, optional): The cryptocurrency exchange to use (default is "binance").
        """
        self.exchange = exchange
        self.cg = CoinGeckoAPI()
        self.coins = [] #Initialize an empty list 
        self.df2 = None  # Initialize an empty DataFrame
        
    def coin_list(self): 
        """
    Retrieve a list of cryptocurrency coins from the specified exchange.

    This method retrieves cryptocurrency data from the CoinGecko API for the specified exchange
    and filters the coins to include only "tether" and "bitcoin".

    Args:
        None

    Returns:
        List of coins that can be traded with "tether" and "bitcoin". 

    Example:
        To fetch and filter coins, call this method as follows:
        >>> crypto_analyzer = CryptoAnalysisClass()
        >>> crypto_analyzer.coin_list()
        Download Completed
    """
        page = 1
        df = pd.DataFrame()  # Initialize an empty DataFrame

        while True:
            # Retrieve ticker data for the specified cryptocurrency exchange (self.exchange) and page number (page)
            data = self.cg.get_exchanges_tickers_by_id(id=self.exchange, page=page)["tickers"]

            if not data:
                print("Download of coin list Completed")
                break
            else:
                # Concatenate DataFrames
                df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
                print(f'Downloading Data...: {page}', end="\r")
                page += 1
                time.sleep(1.5)
                
        # Filter the DataFrame 'df' to include only rows where 'target_coin_id' is 'tether' or 'bitcoin'
        coins_df = df.loc[df["target_coin_id"].isin(["tether", "bitcoin"])]
        
        # Extract the 'coin_id' column from the filtered DataFrame and convert it to a list
        self.coins = list(coins_df["coin_id"])
        
        # Clear the printed line
        print(" " * len("Downloading Data...: 999"), end="\r")
        return self.coins
    
    def panel_data(self, max_coins=5):
        """
        Retrieve time series data for a specified number of cryptocurrencies and store it in the DataFrame.
        
        Args:
            max_coins (int, optional): The maximum number of cryptocurrencies to retrieve data for. Defaults to 5.

        Returns:
            pandas.DataFrame: The DataFrame containing time series data for the specified cryptocurrencies.
        """
        if not self.coins:
            print("Coins list is empty. Calling coin_list() to populate it...")
            print(" " * len("Coins list is empty. Calling coin_list() to populate it...          "), end="\r")

            self.coin_list()
        
        df2 = pd.DataFrame()  # Initialize an empty DataFrame to store time series data
        
        for coin in self.coins[:max_coins]:
            try:
                # Fetch time series data for a specific cryptocurrency (coin)
                time_series = self.cg.get_coin_market_chart_by_id(id=coin, vs_currency="eur", days="max", interval="daily")
                
                # Create a DataFrame (df3) from time series data
                df3 = pd.DataFrame(time_series)
                
                # Extract the "Date", "Market Caps" and "Prices" columns from the df3 
                df3['Date'] = df3['prices'].apply(lambda x: x[0])
                df3['Prices'] = df3['prices'].apply(lambda x: x[1])
                df3['Market Caps'] = df3['market_caps'].apply(lambda x: x[1])
                
                # Convert the "Date" column to datetime format (milliseconds)
                df3["Date"] = pd.to_datetime(df3["Date"], unit="ms")
                
                # Add a new column "symbol" to the DataFrame to indicate the cryptocurrency symbol
                df3["Symbol"] = coin
                
                # Create a new DataFrame with only the "Date," "Prices," and "Market Caps" columns
                df3 = df3.iloc[:-1]  # Remove the last row
                
                df3 = df3[["Symbol",'Date', 'Prices', 'Market Caps']]
                
                # Concatenate (append) df3 to an existing DataFrame (df2) to combine time series data
                df2 = pd.concat([df2, df3], ignore_index=True)
                print(f"Downloading Time Series Data for {coin}...", end="\r")
                time.sleep(3)
                
            except Exception as e:
                print(f"Error downloading data for {coin}: {e}")
                
        # Clear the printed line
        print(" " * len("Downloading Time Series Data for Coin...                      "), end="\r")
        print("Time Series Download completed")
        self.df2 = df2  # Assign the DataFrame to the class attribute
        return df2

![](class3.jpg)

In [7]:
# Create an instance of the CryptoAnalysisClass called my_crypto
my_crypto = CryptoAnalysisClass()
my_crypto.panel_data()

Coins list is empty. Calling coin_list() to populate it...
Download of coin list Completed                                     
Time Series Download completed                                


,Symbol,Date,Prices,Market Caps
0,usd-coin,2018-10-05,0.87,0.00
1,usd-coin,2018-10-06,0.87,0.00
2,usd-coin,2018-10-07,0.87,0.00
3,usd-coin,2018-10-08,0.87,0.00
4,usd-coin,2018-10-09,0.87,0.00
...,...,...,...,...
11998,ethereum,2023-09-08,1539.12,184943293444.14
11999,ethereum,2023-09-09,1528.27,183739142498.73
12000,ethereum,2023-09-10,1526.49,183544103744.53
12001,ethereum,2023-09-11,1508.98,181554943917.67


## Cleaning Panel Data

Finally, we will introduce an additional functionality, we are going to add teh method `clean_panel_data` which performs several data cleaning and preprocessing operations on the time series data stored in the DataFrame self.df2. More precisly, :

 - First checks whether the DataFrame self.df2 is empty (i.e., None). If it's empty, it prints a message indicating that the time series data is empty and calls the panel_data() method to populate it.

- Converts the "Prices" and "Market Caps" columns to numeric".

- Removes duplicate rows from the DataFrame while keeping only the first occurrence of each duplicate.  

- Fill Missing Values.

-  Excludes rows corresponding to coins with a maximum market capitalization of 0. This is achieved by filtering out rows where the maximum market capitalization within each group (grouped by coin symbol) is equal to 0.


In [8]:
import pandas as pd
import time
from pycoingecko import CoinGeckoAPI
class CryptoAnalysisClass():
    """
    A class for analyzing cryptocurrency market data using the CoinGecko API [1].

    Attributes:
        exchange (str): The cryptocurrency exchange to retrieve data from (default is "binance").
        cg (CoinGeckoAPI): An instance of the CoinGeckoAPI class to interact with the CoinGecko API.
        coins (list): An empty list to store cryptocurrency coin IDs (see code below). 
        df2 (pd.DataFrame): An empty DataFrame for storing analysis results (see code below).

    """

    def __init__(self, exchange="binance"):
        """
        Initializes a CryptoAnalysisClass object with the provided exchange.

        Args:
            exchange (str, optional): The cryptocurrency exchange to use (default is "binance").
        """
        self.exchange = exchange
        self.cg = CoinGeckoAPI()
        self.coins = [] #Initialize an empty list 
        self.df2 = None  # Initialize an empty DataFrame
        
    def coin_list(self): 
        """
    Retrieve a list of cryptocurrency coins from the specified exchange.

    This method retrieves cryptocurrency data from the CoinGecko API for the specified exchange
    and filters the coins to include only "tether" and "bitcoin".

    Args:
        None

    Returns:
        List of coins that can be traded with "tether" and "bitcoin". 

    Example:
        To fetch and filter coins, call this method as follows:
        >>> crypto_analyzer = CryptoAnalysisClass()
        >>> crypto_analyzer.coin_list()
        Download Completed
    """
        page = 1
        df = pd.DataFrame()  # Initialize an empty DataFrame

        while True:
            # Retrieve ticker data for the specified cryptocurrency exchange (self.exchange) and page number (page)
            data = self.cg.get_exchanges_tickers_by_id(id=self.exchange, page=page)["tickers"]

            if not data:
                print("Download of coin list Completed")
                break
            else:
                # Concatenate DataFrames
                df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
                print(f'Downloading Data...: {page}', end="\r")
                page += 1
                time.sleep(1.5)
                
        # Filter the DataFrame 'df' to include only rows where 'target_coin_id' is 'tether' or 'bitcoin'
        coins_df = df.loc[df["target_coin_id"].isin(["tether", "bitcoin"])]
        
        # Extract the 'coin_id' column from the filtered DataFrame and convert it to a list
        self.coins = list(coins_df["coin_id"])
        
        # Clear the printed line
        print(" " * len("Downloading Data...: 999"), end="\r")
        return self.coins
    
    def panel_data(self, max_coins=5):
        """
        Retrieve time series data for a specified number of cryptocurrencies and store it in the DataFrame.
        
        Args:
            max_coins (int, optional): The maximum number of cryptocurrencies to retrieve data for. Defaults to 5.

        Returns:
            pandas.DataFrame: The DataFrame containing time series data for the specified cryptocurrencies.
        """
        if not self.coins:
            print("Coins list is empty. Calling coin_list() to populate it...")
            print(" " * len("Coins list is empty. Calling coin_list() to populate it...          "), end="\r")

            self.coin_list()
        
        df2 = pd.DataFrame()  # Initialize an empty DataFrame to store time series data
        
        for coin in self.coins[:max_coins]:
            try:
                # Fetch time series data for a specific cryptocurrency (coin)
                time_series = self.cg.get_coin_market_chart_by_id(id=coin, vs_currency="eur", days="max", interval="daily")
                
                # Create a DataFrame (df3) from time series data
                df3 = pd.DataFrame(time_series)
                
                # Extract the "Date", "Market Caps" and "Prices" columns from the df3 
                df3['Date'] = df3['prices'].apply(lambda x: x[0])
                df3['Prices'] = df3['prices'].apply(lambda x: x[1])
                df3['Market Caps'] = df3['market_caps'].apply(lambda x: x[1])
                
                # Convert the "Date" column to datetime format (milliseconds)
                df3["Date"] = pd.to_datetime(df3["Date"], unit="ms")
                
                # Add a new column "symbol" to the DataFrame to indicate the cryptocurrency symbol
                df3["Symbol"] = coin
                
                # Create a new DataFrame with only the "Date," "Prices," and "Market Caps" columns
                df3 = df3.iloc[:-1]  # Remove the last row
                
                df3 = df3[["Symbol",'Date', 'Prices', 'Market Caps']]
                
                # Concatenate (append) df3 to an existing DataFrame (df2) to combine time series data
                df2 = pd.concat([df2, df3], ignore_index=True)
                print(f"Downloading Time Series Data for {coin}...", end="\r")
                time.sleep(3)
                
            except Exception as e:
                print(f"Error downloading data for {coin}: {e}")
                
        # Clear the printed line
        print(" " * len("Downloading Time Series Data for Coin...                      "), end="\r")
        print("Time Series Download completed")
        self.df2 = df2  # Assign the DataFrame to the class attribute
        return df2
    
    def clean_panel_data(self):
        """
        Clean and preprocess the time series data.

        Returns:
            pd.DataFrame: The cleaned DataFrame.
        """
        if self.df2 is None:
            print("Time series data is empty. Calling panel_data() to populate it...")
            self.panel_data()

        # Convert the "Price" and "Market Caps" columns to numeric
        self.df2["Prices"] = pd.to_numeric(self.df2["Prices"])
        self.df2["Market Caps"] = pd.to_numeric(self.df2["Market Caps"])

        # Remove duplicate rows, keeping only the first occurrence
        self.df2.drop_duplicates(keep="first", inplace=True)
        
        # Filling (only two consecutive missing values) missing values in the "Price" and "Market Caps" columns using forward fill (ffill) method       
        self.df2["Prices"].fillna(method = "ffill",limit=2, inplace=True)
        self.df2["Market Caps"].fillna(method = "ffill",limit=2, inplace=True)
        
        # Set the "Date" column as the index
        self.df2.set_index("Date")

        # Exclude rows corresponding to coins with a maximum market cap of 0        
        self.df2.loc[~self.df2["Symbol"].isin(self.df2.groupby("Symbol")['Market Caps'].max() == 0)].copy

        # Clear the printed line
        print(" " * len("Time Series Download completed"), end="\r")
        print("Data are Cleaned")
        return self.df2


![](class4.jpg)

In [9]:
# Create an instance of the CryptoAnalysisClass called my_crypto
my_crypto = CryptoAnalysisClass()
my_crypto.panel_data(max_coins=100)
my_crypto.clean_panel_data()

Coins list is empty. Calling coin_list() to populate it...
Download of coin list Completed                                     
Time Series Download completed                                
Data are Cleaned              


,Symbol,Date,Prices,Market Caps
0,usd-coin,2018-10-05,0.87,0.00
1,usd-coin,2018-10-06,0.87,0.00
2,usd-coin,2018-10-07,0.87,0.00
3,usd-coin,2018-10-08,0.87,0.00
4,usd-coin,2018-10-09,0.87,0.00
...,...,...,...,...
167361,iexec-rlc,2023-09-08,1.03,74298225.75
167362,iexec-rlc,2023-09-09,1.02,73973036.71
167363,iexec-rlc,2023-09-10,0.99,71788704.03
167364,iexec-rlc,2023-09-11,0.96,69738771.56


## Conclusion

Cryptocurrency market data analysis is crucial for making informed investment decisions and gaining insights into this rapidly evolving financial landscape. With the CryptoAnalysisClass we've created in this tutorial, we've begun building a powerful tool to retrieve, clean, and preprocess cryptocurrency data. In the next tutorial, we will expand its functionality by adding features like calculating various metrics such as the Sharpe ratio, drawdown, and more!

## Reference:
[1]	“ChatGPT was used to write the docstring of the CryptoAnalysisClass class.” https://chat.openai.com (accessed Sep. 11, 2023).  

[2]	“Automated Cryptocurrency Portfolio Investing with Python A-Z,” Udemy. https://www.udemy.com/course/automated-cryptocurrency-portfolio-investing-with-python/ (accessed Apr. 01, 2023).



![](oop2.jpg)